In [55]:
pip install parsel fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=04503122c241c68d1e1ab6c6703af31028370fa58e41e79b79aef3e47b1990ca
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [71]:
# !pip install parsel
import requests
from parsel import Selector
from fake_useragent import UserAgent
from urllib.parse import urljoin

import json


def crawl(asin_no: str) -> dict:
    base_url: str = 'https://www.amazon.cn/dp/' + asin_no + '/ref=z_cn'
    headers: dict = {
        # "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
        "User-Agent": UserAgent().random,
        'Cookie': 'session-id=457-4712657-0321906; ubid-acbcn=458-7940515-9599013; x-acbcn="VZU?pPyRMUhNogIOVS?2BX?zroShiKcjr6AXEpGM5If?buubfVZyXWm1dEpdP5CY"; at-main=Atza|IwEBIPqnFPANuSQ54XEOYnk-Wh1HdOKhB0hqCmhiSKdF4v6qLoNrSz1qeaghOPOv-KloCnqFs7gQSe-JDWbrNHM3jX1UeRWzQW_nXFItpl52c1DNrfGCdRJ2iWHk0TQy_L-ENijF7s2vfWuFxtRq38J3a4_QughA92rg3dojEItjQa_nqgqxKTkby7CSZbYs5--PSLHg0GKFUOcmvMQgQluUHgRCzBTUJBR9SxtS2Dc-NHWorA; sess-at-main="vDi1Ir2E4+MsC9cjP92cDaQQ3G9Nfx/NUkYFhOb/dG0="; sst-main=Sst1|PQHK7-dq45XYrUuGSix2lAlhCRyiW2NWVFuna74UkYXNpjNlyOKDJBCmQr7J0uPV6tgFWaMi6WTaXTfU4fy2q-vDO0hXtckM8V04J1DtQ5fZUJ6Rg7pB10gqDL8NMTYAv7FelKjzdkZ9XYocpzycU2RL8MGj1vWzgD6a6s8-Vl2w7bX4nxLsQWHvFaHkBFaP7FBLIHcy-wQRu7ffgS1R52ivCFL3XUsj8wdEeb0GgUk_Lj2YDHx0KtnODxZCZxmQUzr6cWpIzoMZQ9bHAZ4NpLGy6A-xV1JBbokbWkwVF1Vf9Ns; lc-acbcn=zh_CN; i18n-prefs=CNY; s_cc=true; s_vnum=2090596794466&vn=1; s_sq=[[B]]; s_nr=1658596844424-New; s_dslv=1658596844426; s_ppv=43; session-token=vj9pHfWT4XRBuZ+x6sqqs2aStssRhtp1YERRRqhmmJx+Y5Hcf6McmtFPPiIhkpYt/bMpWj6t1X+gmxqtoeg4Am07DlwqDZT2xw9EYvpDTwJ4EsNqF28EHMa8NMVSiR0vw+dOjJUP3X/AGbf7XGnenYLfleplsD7tFKIaonrCK72tx+BXymPnQx7iazKT0PD6D9DPF+tfo5bPQmL80Cc4kGiEtOxhxIsw; session-id-time=2082729601l; csm-hit=tb:9NCA8D53SB29HAZYH14G+sa-XAZ077H79F9W4YCZ0EJX-1EHD48SB2KK3R9PP4MH1|1658908448718&t:1658908448719&adb:adblk_no'
    }

    resp = requests.get(base_url, headers=headers)
    # print(str(resp))

    content = resp.content.decode('utf-8')
    # print(str(content))

    select = Selector(text=content)

    # print(str(select))

    productTitle = select.xpath('//*[@id="productTitle"]/text()').extract_first().strip()
    # ags_shipping_import_fee = select.xpath('//*[@id="ags_shipping_import_fee"]/text()').extract_first()
    print("productTitle: " + productTitle)
    # print("ags_shipping_import_fee: "+ags_shipping_import_fee)

    # merchant_id = select.xpath('//*[@id="merchantID"]/text()').extract_first().strip()
    merchant_id = select.css('#merchantID').re_first(
        '<input type="hidden" id="merchantID" name="merchantID" value="(.*?)">')
    # merchant_id = selector.css('.item-0').re_first('<span class="bold">(.*?)</span>')
    print("merchant_id: " + merchant_id)

    # asin_info = {
    #     "productTitle": select.xpath("/text()"),
    #     "ags_shipping_import_fee": select.xpath("/text()")
    # }

    asin_info = {
        "productTitle": select.xpath('//*[@id="productTitle"]/text()').extract_first().strip(),
        # "ags_shipping_import_fee": select.xpath('//*[@id="ags_shipping_import_fee"]/text()').extract_first().strip()
        "ags_shipping_import_fee": ags_shipping_import_fee(asin_no, merchant_id)
    }

    return asin_info


def ags_shipping_import_fee(asin_no: str, merchant_id: str) -> float:
    # base_url: str = 'https://data.amazon.cn/api/marketplaces/AAHKV2X7AFYLW/products/' + asin_no + '?merchantId=' + merchant_id
    base_url: str = 'https://data.amazon.cn/api/marketplaces/AAHKV2X7AFYLW/products/' + asin_no + '?merchantId=A3CQWPW49OI3BQ'
    headers: dict = {
        # "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
        # "Accept": "application/vnd.com.amazon.api+json; type=\"product/v2\"; expand=\"buyingOptions[].importDetails(product.import-details/v1),buyingOptions[].globalStore(product.global-store/v1)\"",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN",
        "DNT":"1",
        "Host":"data.amazon.cn",
        "Origin":"www.amazon.cn",
        "Referer":"www.amazon.cn",
        "X-API-CSRF-TOKEN": "1@g0dZsyVyOIZbol4jMaJqoRZp55qHnLQXHam+x9ynsDQCAAAAAQAAAABi4O9ocmF3AAAAABVX8CwXqz42z+J7i/ABqA==@NLD_5740BY",
        "sec-ch-ua": "\".Not/A)Brand\";v=\"99\", \"Google Chrome\";v=\"103\", \"Chromium\";v=\"103\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "Windows",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",

    "Connection": "keep-alive",
    "Content-Type":"application/json",
    # 'Accept':'*/*',
    # 'Accept-Language': 'zh-CN,zh;q=0.9',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    # 'accept-encoding': 'gzip, deflate, br',
    # # 'accept-language': 'zh-CN,zh;q=0.9',
    # 'cache-control': 'max-age=0',
    # 'downlink': '8',
    # 'ect': '4g',
    # 'rtt': '250',
    # 'Cookie': "session-id=257-3500989-3695223; i18n-prefs=GBP; ubid-acbuk=257-5950834-2508848; x-wl-uid=1bEcLG2b03/1tAwPJNyfuRH+U7J9ZaPYejSBR4HXKuYQPJtLhQbDYyO/GOMypGKXqZrG7qBkS0ng=; session-token=x04EF8doE84tE+6CXYubsjmyob/3M6fdmsQuqzD0jwl/qGdO5aRc2eyhGiwoD0TFzK1rR/yziHsDS4v6cdqT2DySFXFZ9I5OHEtgufqBMEyrA0/Scr87KKA+GWOjfVmKRuPCqOGaixZQ6AIjU3e2iFOdM+3v90NeXFI3cazZcd6x9TYCy9b5u9V8zR7ePbdP; session-id-time=2082758401l; csm-hit=tb:MAA188S1G57TNTH6HQCZ+s-T9EGT4C8FC8J74X5T7CY|1594212767446&t:1594212767446&adb:adblk_no",
    # 'upgrade-insecure-requests': '1',
    # 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
        "User-Agent": UserAgent().random,
        'Cookie': 'session-id=457-4712657-0321906; ubid-acbcn=458-7940515-9599013; x-acbcn="VZU?pPyRMUhNogIOVS?2BX?zroShiKcjr6AXEpGM5If?buubfVZyXWm1dEpdP5CY"; at-main=Atza|IwEBIPqnFPANuSQ54XEOYnk-Wh1HdOKhB0hqCmhiSKdF4v6qLoNrSz1qeaghOPOv-KloCnqFs7gQSe-JDWbrNHM3jX1UeRWzQW_nXFItpl52c1DNrfGCdRJ2iWHk0TQy_L-ENijF7s2vfWuFxtRq38J3a4_QughA92rg3dojEItjQa_nqgqxKTkby7CSZbYs5--PSLHg0GKFUOcmvMQgQluUHgRCzBTUJBR9SxtS2Dc-NHWorA; sess-at-main="vDi1Ir2E4+MsC9cjP92cDaQQ3G9Nfx/NUkYFhOb/dG0="; sst-main=Sst1|PQHK7-dq45XYrUuGSix2lAlhCRyiW2NWVFuna74UkYXNpjNlyOKDJBCmQr7J0uPV6tgFWaMi6WTaXTfU4fy2q-vDO0hXtckM8V04J1DtQ5fZUJ6Rg7pB10gqDL8NMTYAv7FelKjzdkZ9XYocpzycU2RL8MGj1vWzgD6a6s8-Vl2w7bX4nxLsQWHvFaHkBFaP7FBLIHcy-wQRu7ffgS1R52ivCFL3XUsj8wdEeb0GgUk_Lj2YDHx0KtnODxZCZxmQUzr6cWpIzoMZQ9bHAZ4NpLGy6A-xV1JBbokbWkwVF1Vf9Ns; lc-acbcn=zh_CN; i18n-prefs=CNY; s_cc=true; s_vnum=2090596794466&vn=1; s_sq=[[B]]; s_nr=1658596844424-New; s_dslv=1658596844426; s_ppv=43; session-token=rapDMzM4cXIxtLB/v2cGLD8ANRA1sXxDvTV+tGKniGDKsOyvOQSZ7vrPGONw0f62+1Vd+mGwADlv2TlGDAFtotctxE3z9zbpLqMTzCDnatrk78rwXRUdVHFTYlq5mg+pGFdlbRAkvAviw5Kazb1jaxRiJd8fjuIzqgkA/mcsSHF8rwhyZKYht6jnhTlY+6ngdqWe3J7Qabha8WVQBhzm3UK12ycjou3s; session-id-time=2082787201l'

    }

    resp = requests.get(base_url, headers=headers)
    content = resp.content.decode('utf-8')

    print("content:"+content)

    resp_json = json.loads(content)
    ags_shipping_import_fee = resp_json["entity"]['buyingOptions'](0)['importDetails']['entity']['fee']['amount']

    return float(ags_shipping_import_fee)


if __name__ == '__main__':
    product_info: dict = crawl(asin_no='B0050N5DPY')
    print(product_info)


productTitle: Clarks Hamble Oak 20350674 女士系带鞋
merchant_id: 
content:{"resource":{"url":"/api/marketplaces/AAHKV2X7AFYLW/products/B0050N5DPY","types":[]},"type":"error/v1","entity":{"status":415,"code":"415.1","details":{"message":"Sorry, We are experiencing issues right now.","url":"https://api.amazon.com/shop"},"internalInfo":{"message":"The MediaType supplied in the HTTP requests 'Accept' header was invalid.","frontend":{"requestId":"GH9PCM4EEGBHOYL6ELJP","originalRequestId":"8ECB6ZH708JFKGQPP2E7","time":"2022-07-27T08:04:44.231Z","host":"fu+IX8hoxuXTRxwWh4QQbUP97yPWC/ZcFFSgIfstbXz4htoHQmHEqc6uFngVvojl9N2UVTVhqtmowMU4P1lfzg=="}}}}


KeyError: ignored

In [ ]:
if __name__ == '__main__':
    # 设置get直接返回，不再等待界面加载完成
    desired_capabilities = DesiredCapabilities.CHROME
    desired_capabilities["pageLoadStrategy"] = "none"
    chrome_options = webdriver.ChromeOptions()
    # 无窗口模式
    # chrome_options.add_argument('--headless')
    # 禁止硬件加速，避免严重占用cpu
    chrome_options.add_argument('--disable-gpu')
    # 关闭安全策略
    chrome_options.add_argument("disable-web-security")
    # 禁止图片加载
    chrome_options.add_experimental_option('prefs', {'profile.managed_default_content_settings.images': 2})
    # 隐藏"Chrome正在受到自动软件的控制
    chrome_options.add_argument('disable-infobars')
    # 设置开发者模式启动，该模式下webdriver属性为正常值
    chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
    # 模拟浏览器
    chrome_options.add_argument(
       'user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"')
    driver = webdriver.Chrome(options=chrome_options)
    # 返回驱动等待的变量
    wait = WebDriverWait(driver, 20)
    # driver.maximize_window()
    write_row = 1
    workbook = xlsxwriter.Workbook('E:/耳机商品列表—英文.xlsx')
    worksheet = workbook.add_worksheet()  # 创建一个sheet
    excel_title = ["标题", "价格", "位置", "商品网址"]
    worksheet.write_row('A1', excel_title)  # title 写入Excel
    print(time.strftime("start %Y-%m-%d %H:%M:%S", time.localtime()))
    # 固定搜索内容，变化的只有页面
    search_page_url = "https://www.amazon.com/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A172541&dc&page={}&language=en_US&currency=USD&qid=1609918038&ref=sr_pg_2"
    for i in range(1, 500):
        print("正在爬取", search_page_url.format(i))
        driver.get(search_page_url.format(i))
        time.sleep(3)
        # css选择器，返回结果存在跳出，异常报错
        try:
            wait.until(ec.presence_of_element_located((By.ID, "be")))
            isEnd = parse_detail(driver.page_source, i)
            if isEnd:
              break
        except:
            print("url: " + search_page_url.format(i) + "获取失败")
            pass
    print(time.strftime("end %Y-%m-%d %H:%M:%S", time.localtime()))
    driver.quit()
    workbook.close()


In [ ]:
pip install selenium urllib3

In [ ]:
# !pip install selenium urllib3
from selenium import webdriver
import time
from lxml import etree
import pandas as pd
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait


def spider_data(num, aa):
    # 让浏览器 不加载图片、css样式
    chrome_options = Options()
    prefs = {
        'profile.managed_default_content_settings.images': 2,
        'permissions.default.stylesheet': 2
    }
    chrome_options.add_experimental_option('prefs', prefs)
    chrome_options.add_argument('--headless')

    # desired_capabilities = DesiredCapabilities.CHROME  # 修改页面加载策略
    # desired_capabilities["pageLoadStrategy"] = "none"  # 注释这两行会导致最后输出结果的延迟，即等待页面加载完成再输出

    # driver = webdriver.Chrome(r'C:\Lee\pycharm\selenium_chorme插件\chromedriver.exe', chrome_options=chrome_options)

    # driver = webdriver.Chrome(r'C:\Lee\pycharm\selenium_chorme插件\chromedriver.exe')
    # driver = webdriver.Chrome('/content/sample_data/chromedriver', chrome_options=chrome_options)
    driver = webdriver.Chrome('sample_data/chromedriver', chrome_options=chrome_options)
    
    url = 'https://www.amazon.com/dp/' + num

    asin_s = []
    stock_s = []  # 库存数据
    stock_mark = []  # 是否限购标记

    asin_s.append(num)

    # 隐式等待
    # driver.implicitly_wait(5)
    driver.get(url)
    try:
        element1 = WebDriverWait(driver, 4, 0.5).until(
            expected_conditions.presence_of_element_located((By.ID, 'add-to-cart-button')))
        # # 1.获取页面对象的 加入购物车 按钮，并点击
        # # driver.find_element_by_id("add-to-cart-button").click()
        element1.click()

        # 2.找到亚马逊右上角的 含有商品的购物车图标，并点击
        driver.find_element_by_id('nav-cart-count').click()

        # 3.找到输入框的定位元素，并点击
        driver.find_element_by_id('a-autoid-0-announce').click()

        # 4.选择下拉框最底部的 最大值，并点击
        driver.find_element_by_id('quantity_10').click()

        try:
            # 5.定位输入框，并输入最大值999
            driver.find_element_by_name('quantityBox').send_keys('999')
            print('已输入数字')
            time.sleep(1)
            # 6.点击 确定按钮
            driver.find_element_by_id('a-autoid-1-announce').click()
            time.sleep(2)
            html = driver.page_source
            # print(html)
            a = etree.HTML(html)

            # 不同页面的 目标数据 位置不一样
            num1 = a.xpath('//div[@class="a-popover-content"]/text()')
            num2 = a.xpath('//div[@class="a-alert-content"]/span[@class="a-size-base"]/text()')
            result = num1 or num2 if num1 or num2 else ' '
            print(result)
            if '只有' in result[0]:
                stock_s.append(result[0].replace('\n此商品在您选择的卖家处只有', '').replace('件。到', ''))
                stock_mark.append('False')
            elif '限制' in result[0]:
                stock_s.append(result[0].replace('\n此卖家对该商品的数量限制为每位客户', '').replace('件。到', ''))
                stock_mark.append('True')
            else:
                stock_s.append(' ')
                stock_mark.append('Not updated')

            # 保存数据
            data = {'target_asin': asin_s,
                    'stock_num': stock_s,
                    'stock_mark': stock_mark}
            df = pd.DataFrame(data, columns=['target_asin', 'stock_num', 'stock_mark'])
            df.to_csv('stock_data.csv', mode='a', index=False, encoding='utf_8_sig')
            print('爬取成功：', str(aa))

            # 关闭页面
            driver.quit()

        except Exception as error:
            print('fail：', error)
            # 关闭页面
            driver.quit()
    except:
        print(num, '缺货中')
        stock_s.append(0)
        stock_mark.append('-')

        # 保存数据
        data = {'target_asin': asin_s,
                'stock_num': stock_s,
                'stock_mark': stock_mark}
        df = pd.DataFrame(data, columns=['target_asin', 'stock_num', 'stock_mark'])
        df.to_csv('stock_data.csv', mode='a', index=False, encoding='utf_8_sig')

        # 关闭页面
        driver.quit()


if __name__ == '__main__':
    # 这里需要用excel记录一列需要爬取的asin数据，列字段为asin
    # df = pd.read_excel(r'C:\Users\sparkx\Desktop\asin_data.xlsx')
    aa = 0
    # for num in df['asin']:
    #     aa += 1
    #     spider_data(num, aa)
        # time.sleep(1)
    
    num='B0050N5DPY'
    spider_data(num, aa)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: use options instead of chrome_options


WebDriverException: ignored

In [ ]:
from selenium import webdriver
from lxml import etree
from selenium.webdriver.common.by import By  # 按照什么方式查找，By.ID,By.CSS_SELECTOR
from selenium.webdriver.common.keys import Keys  # 键盘按键操作
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait  # 等待页面加载某些元素
import csv
import time
all_list = []
class Amazon(object):
    def __init__(self):
        self.url = 'https://www.amazon.com/s?k=%E7%A7%AF%E6%9C%A8&ref=nb_sb_noss'
        self.browser = webdriver.Chrome()

    def __del__(self):
        self.browser.close()

    def get_html(self):
        time.sleep(3)
        text = self.browser.page_source
        html = etree.HTML(text)

        next_url = WebDriverWait(self.browser, 10).until(EC.presence_of_element_located((By.XPATH,'//li[@class="a-last"]')))
        is_next = next_url.get_attribute('xh-highlight')
        urls = html.xpath('//a[@class="a-link-normal s-no-outline"]/@href')
        title = html.xpath('//span[@class="a-size-base-plus a-color-base a-text-normal"]/text()')
        pinfen = html.xpath('//span[@class="a-icon-alt"]/text()')
        rating_value = html.xpath('//span[@class="a-size-base"]/text()')
        prcie = html.xpath('//span[@class="a-offscreen"]/text()')
        for i in range(0,len(urls)):
            try:
                print(title[i], pinfen[i], rating_value[i], urls[i],prcie[i])
                list = [title[i], pinfen[i], rating_value[i], urls[i],prcie[i]]
                all_list.append(list)
            except Exception as e:
                pass
        # 使用csv存储数据
        headers = ["标题",'评分','评论数','链接','价格']
        with open('amazon.csv', 'w', newline='', encoding='utf-8')as f:
            f_csv = csv.writer(f)
            f_csv.writerow(headers)
            f_csv.writerows(all_list)

        return next_url,is_next

    def main(self):
        time.sleep(2)
        self.browser.get(self.url)
        while 1:
            next_url,is_next = self.get_html()
            if is_next is None:
                break
            next_url.click()


if __name__ == '__main__':
    amazon_spider = Amazon()
    amazon_spider.main()



WebDriverException: ignored

In [ ]:
# pip install selenium
chmod +x sample_data/chromedriver

In [ ]:
!pip install kora -q
from kora.selenium import wd
# from selenium import webdriver
from lxml import etree
from selenium.webdriver.common.by import By  # 按照什么方式查找，By.ID,By.CSS_SELECTOR
from selenium.webdriver.common.keys import Keys  # 键盘按键操作
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait  # 等待页面加载某些元素
import csv
import time
all_list = []
class Amazon(object):
    def __init__(self):
        self.url = 'https://www.amazon.com/s?k=%E7%A7%AF%E6%9C%A8&ref=nb_sb_noss'
        # self.browser = webdriver.Chrome('sample_data/chromedriver')
        # self.browser = webdriver.Chrome(r'/content/sample_data/chromedriver')
        self.browser = wd

    def __del__(self):
        self.browser.close()

    def get_html(self):
        time.sleep(3)
        text = self.browser.page_source
        html = etree.HTML(text)

        # next_url = WebDriverWait(self.browser, 10).until(EC.presence_of_element_located((By.XPATH,'//li[@class="a-last"]')))
        # is_next = next_url.get_attribute('xh-highlight')
        urls = html.xpath('//a[@class="a-link-normal s-no-outline"]/@href')
        title = html.xpath('//span[@class="a-size-base-plus a-color-base a-text-normal"]/text()')
        pinfen = html.xpath('//span[@class="a-icon-alt"]/text()')
        rating_value = html.xpath('//span[@class="a-size-base"]/text()')
        prcie = html.xpath('//span[@class="a-offscreen"]/text()')
        for i in range(0,len(urls)):
            try:
                print(title[i], pinfen[i], rating_value[i], urls[i],prcie[i])
                list = [title[i], pinfen[i], rating_value[i], urls[i],prcie[i]]
                all_list.append(list)
            except Exception as e:
                pass
        # 使用csv存储数据
        headers = ["标题",'评分','评论数','链接','价格']
        with open('amazon.csv', 'w', newline='', encoding='utf-8')as f:
            f_csv = csv.writer(f)
            f_csv.writerow(headers)
            f_csv.writerows(all_list)

        return next_url,is_next

    def main(self):
        time.sleep(2)
        self.browser.get(self.url)
        while 1:
            next_url,is_next = self.get_html()
            if is_next is None:
                break
            next_url.click()


if __name__ == '__main__':
    amazon_spider = Amazon()
    amazon_spider.main()



In [ ]:
# !pip install kora - q
from kora.selenium import wd
# from selenium import webdriver
from lxml import etree
from selenium.webdriver.common.by import By  # 按照什么方式查找，By.ID,By.CSS_SELECTOR
from selenium.webdriver.common.keys import Keys  # 键盘按键操作
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait  # 等待页面加载某些元素
import csv
import time

all_list = []


class Amazon(object):
    def __init__(self):
        self.url = 'https://www.amazon.cn/dp/B0050N5DPY/ref=z_cn'
        # self.browser = webdriver.Chrome('sample_data/chromedriver')
        # self.browser = webdriver.Chrome(r'/content/sample_data/chromedriver')
        self.browser = wd

    def __del__(self):
        self.browser.close()

    def get_html(self):
        time.sleep(3)
        text = self.browser.page_source
        html = etree.HTML(text)

        productTitle = html.xpath('//*[@id="productTitle"]/text()')
        print("productTitle: "+ str(productTitle))
        urls = html.xpath('//a[@class="a-link-normal s-no-outline"]/@href')
        title = html.xpath('//span[@class="a-size-base-plus a-color-base a-text-normal"]/text()')
        pinfen = html.xpath('//span[@class="a-icon-alt"]/text()')
        rating_value = html.xpath('//span[@class="a-size-base"]/text()')
        prcie = html.xpath('//span[@class="a-offscreen"]/text()')
        for i in range(0, len(urls)):
            try:
                print(title[i], pinfen[i], rating_value[i], urls[i], prcie[i])
                list = [title[i], pinfen[i], rating_value[i], urls[i], prcie[i]]
                all_list.append(list)
            except Exception as e:
                pass

    def main(self):
        time.sleep(2)
        self.browser.get(self.url)
        result = self.get_html()
        print(result)


if __name__ == '__main__':
    amazon_spider = Amazon()
    amazon_spider.main()


In [ ]:
# !pip install selenium urllib3
from kora.selenium import wd
# from selenium import webdriver
import time
from lxml import etree
import pandas as pd
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait


def spider_data(num, aa):
    # 让浏览器 不加载图片、css样式
    chrome_options = Options()
    prefs = {
        'profile.managed_default_content_settings.images': 2,
        'permissions.default.stylesheet': 2
    }
    chrome_options.add_experimental_option('prefs', prefs)
    chrome_options.add_argument('--headless')

    desired_capabilities = DesiredCapabilities.CHROME  # 修改页面加载策略
    desired_capabilities["pageLoadStrategy"] = "none"  # 注释这两行会导致最后输出结果的延迟，即等待页面加载完成再输出

    # driver = webdriver.Chrome(r'C:\Lee\pycharm\selenium_chorme插件\chromedriver.exe', chrome_options=chrome_options)

    # driver = webdriver.Chrome(r'C:\Lee\pycharm\selenium_chorme插件\chromedriver.exe')
    driver = wd

    url = 'https://www.amazon.com/dp/' + num
    url ='https://www.amazon.cn/dp/' + num + '/ref=z_cn'

    asin_s = []
    stock_s = []  # 库存数据
    stock_mark = []  # 是否限购标记

    asin_s.append(num)

    # 隐式等待
    driver.implicitly_wait(5)
    driver.get(url)
    try:
        element1 = WebDriverWait(driver, 4, 0.5).until(
            expected_conditions.presence_of_element_located((By.ID, 'add-to-cart-button')))
        # # 1.获取页面对象的 加入购物车 按钮，并点击
        # # driver.find_element_by_id("add-to-cart-button").click()
        element1.click()

        # 2.找到亚马逊右上角的 含有商品的购物车图标，并点击
        driver.find_element_by_id('nav-cart-count').click()

        # 3.找到输入框的定位元素，并点击
        driver.find_element_by_id('a-autoid-0-announce').click()

        # 4.选择下拉框最底部的 最大值，并点击
        driver.find_element_by_id('quantity_10').click()

        try:
            # 5.定位输入框，并输入最大值999
            driver.find_element_by_name('quantityBox').send_keys('999')
            print('已输入数字')
            time.sleep(1)
            # 6.点击 确定按钮
            driver.find_element_by_id('a-autoid-1-announce').click()
            time.sleep(2)
            html = driver.page_source
            # print(html)
            a = etree.HTML(html)

            # 不同页面的 目标数据 位置不一样
            num1 = a.xpath('//div[@class="a-popover-content"]/text()')
            num2 = a.xpath('//div[@class="a-alert-content"]/span[@class="a-size-base"]/text()')
            result = num1 or num2 if num1 or num2 else ' '
            print(result)
            if '只有' in result[0]:
                stock_s.append(result[0].replace('\n此商品在您选择的卖家处只有', '').replace('件。到', ''))
                stock_mark.append('False')
            elif '限制' in result[0]:
                stock_s.append(result[0].replace('\n此卖家对该商品的数量限制为每位客户', '').replace('件。到', ''))
                stock_mark.append('True')
            else:
                stock_s.append(' ')
                stock_mark.append('Not updated')

            # 保存数据
            data = {'target_asin': asin_s,
                    'stock_num': stock_s,
                    'stock_mark': stock_mark}
            df = pd.DataFrame(data, columns=['target_asin', 'stock_num', 'stock_mark'])
            df.to_csv('stock_data.csv', mode='a', index=False, encoding='utf_8_sig')
            print('爬取成功：', str(aa))

            # 关闭页面
            driver.quit()

        except Exception as error:
            print('fail：', error)
            # 关闭页面
            driver.quit()
    except:
        print(num, '缺货中')
        stock_s.append(0)
        stock_mark.append('-')

        # 保存数据
        data = {'target_asin': asin_s,
                'stock_num': stock_s,
                'stock_mark': stock_mark}
        df = pd.DataFrame(data, columns=['target_asin', 'stock_num', 'stock_mark'])
        df.to_csv('stock_data.csv', mode='a', index=False, encoding='utf_8_sig')

        # 关闭页面
        driver.quit()


if __name__ == '__main__':
    # 这里需要用excel记录一列需要爬取的asin数据，列字段为asin
    # df = pd.read_excel(r'C:\Users\sparkx\Desktop\asin_data.xlsx')
    aa = 0
    # for num in df['asin']:
    #     aa += 1
    #     spider_data(num, aa)
        # time.sleep(1)

    num='B0050N5DPY'
    spider_data(num, aa)


In [ ]:
pip install fake_useragent

In [ ]:
import requests
from fake_useragent import UserAgent
import re,os,time
from lxml import etree


def ua():
    ua=UserAgent()
    headers={
        'User-Agent':ua.random,
        'Cookie': 'session-id=457-4712657-0321906; ubid-acbcn=458-7940515-9599013; x-acbcn="VZU?pPyRMUhNogIOVS?2BX?zroShiKcjr6AXEpGM5If?buubfVZyXWm1dEpdP5CY"; at-main=Atza|IwEBIPqnFPANuSQ54XEOYnk-Wh1HdOKhB0hqCmhiSKdF4v6qLoNrSz1qeaghOPOv-KloCnqFs7gQSe-JDWbrNHM3jX1UeRWzQW_nXFItpl52c1DNrfGCdRJ2iWHk0TQy_L-ENijF7s2vfWuFxtRq38J3a4_QughA92rg3dojEItjQa_nqgqxKTkby7CSZbYs5--PSLHg0GKFUOcmvMQgQluUHgRCzBTUJBR9SxtS2Dc-NHWorA; sess-at-main="vDi1Ir2E4+MsC9cjP92cDaQQ3G9Nfx/NUkYFhOb/dG0="; sst-main=Sst1|PQHK7-dq45XYrUuGSix2lAlhCRyiW2NWVFuna74UkYXNpjNlyOKDJBCmQr7J0uPV6tgFWaMi6WTaXTfU4fy2q-vDO0hXtckM8V04J1DtQ5fZUJ6Rg7pB10gqDL8NMTYAv7FelKjzdkZ9XYocpzycU2RL8MGj1vWzgD6a6s8-Vl2w7bX4nxLsQWHvFaHkBFaP7FBLIHcy-wQRu7ffgS1R52ivCFL3XUsj8wdEeb0GgUk_Lj2YDHx0KtnODxZCZxmQUzr6cWpIzoMZQ9bHAZ4NpLGy6A-xV1JBbokbWkwVF1Vf9Ns; lc-acbcn=zh_CN; i18n-prefs=CNY; s_cc=true; s_vnum=2090596794466&vn=1; s_sq=[[B]]; s_nr=1658596844424-New; s_dslv=1658596844426; s_ppv=43; session-token=rapDMzM4cXIxtLB/v2cGLD8ANRA1sXxDvTV+tGKniGDKsOyvOQSZ7vrPGONw0f62+1Vd+mGwADlv2TlGDAFtotctxE3z9zbpLqMTzCDnatrk78rwXRUdVHFTYlq5mg+pGFdlbRAkvAviw5Kazb1jaxRiJd8fjuIzqgkA/mcsSHF8rwhyZKYht6jnhTlY+6ngdqWe3J7Qabha8WVQBhzm3UK12ycjou3s; session-id-time=2082787201l'
             }
    return headers


#保存txt
def tx(id,text,path):
    print(f"正在保存商品数据..")
    with open(f'{path}{id}.txt','w') as f:
        f.write(text)
    print(f">>>保存商品数据成功！")


#下载图片
def down(img_url,img_name,path):
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    print(f"下载{img_name}图片..")
    r=requests.get(img_url,headers=headers,timeout=10)
    with open(f'{path}{img_name}','wb') as f:
        f.write(r.content)
        print(f">>>下载{img_name}图片完成！")
    time.sleep(2)


def get_shopping(id):
    url=f"https://www.amazon.cn/dp/{id}"
    html=requests.get(url,headers=ua(),timeout=10).content.decode('utf-8')
    print(str(html))

    req=etree.HTML(html)

    productTitle = req.xpath('//*[@id="productTitle"]/text()').extract_first().strip()
    # ags_shipping_import_fee = select.xpath('//*[@id="ags_shipping_import_fee"]/text()').extract_first()
    print("productTitle: " + productTitle)

    time.sleep(3)
    title=re.findall(r'"立即购买：(.+?)",',html,re.S)[0] #B&O PLAY by Bang & Olufsen Beoplay P6 便携式扬声器1140026 黑色
    print(title)
    path=f'{title}/'
    os.makedirs(path,exist_ok=True) #创建目录
    req=etree.HTML(html)
    price=req.xpath('//span[@id="priceblock_ourprice"]/text()')[0]
    print(price)
    text='%s%s%s%s%s'%(url,'\n',title,'\n',price)
    tx(id, text, path)
    imgs=req.xpath('//span[@class="a-button-text"]/img/@src')
    for img in imgs:
        if 'jpg' in img:
            imgurl=img.split('._')[0]
            img_url =f'{imgurl}.jpg'
            img_name = img_url.split('/')[-1]
            print(img_url,img_name)
            down(img_url, img_name, path)




if __name__=='__main__':
    # id=input("请输入要采集的商品id（比如：B00C3YADSK）：")
    id='B0050N5DPY'
    get_shopping(id)


In [ ]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def run():
    option = Options()
    option.add_experimental_option("detach", True)
    option.add_experimental_option('excludeSwitches', ['enable-automation'])
    option.add_argument('--disable-blink-features=AutomationControlled')
    web = Chrome(options=option)
    web.get('https://www.amazon.cn/dp/B0050N5DPY/ref=z_cn')
    time.sleep(2)
    # 获取商品名
    product_name = web.find_element(By.XPATH, '//*[@id="productTitle"]').text
    print('商品名：',product_name)
    time.sleep(2)
    web.find_element(By.XPATH, '//*[@id="native_dropdown_selected_size_name"]').click()
    time.sleep(1)
    web.find_element(By.XPATH, '//*[@id="native_size_name_11"]').click()
    time.sleep(3)
    taxes = web.find_element(By.XPATH, '//*[@id="ags_shipping_import_fee"]').text
    print('税费：',taxes)

if __name__ == '__main__':
    run()

In [ ]:
!pip install parsel fake_useragent
import requests
from parsel import Selector
from fake_useragent import UserAgent
from urllib.parse import urljoin

import json


def crawl(asin_no: str) -> dict:
    base_url: str = 'https://www.amazon.cn/dp/' + asin_no + '/ref=z_cn'
    headers: dict = {
        # "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
        "User-Agent": UserAgent().random,
        'Cookie': 'session-id=457-4712657-0321906; ubid-acbcn=458-7940515-9599013; x-acbcn="VZU?pPyRMUhNogIOVS?2BX?zroShiKcjr6AXEpGM5If?buubfVZyXWm1dEpdP5CY"; at-main=Atza|IwEBIPqnFPANuSQ54XEOYnk-Wh1HdOKhB0hqCmhiSKdF4v6qLoNrSz1qeaghOPOv-KloCnqFs7gQSe-JDWbrNHM3jX1UeRWzQW_nXFItpl52c1DNrfGCdRJ2iWHk0TQy_L-ENijF7s2vfWuFxtRq38J3a4_QughA92rg3dojEItjQa_nqgqxKTkby7CSZbYs5--PSLHg0GKFUOcmvMQgQluUHgRCzBTUJBR9SxtS2Dc-NHWorA; sess-at-main="vDi1Ir2E4+MsC9cjP92cDaQQ3G9Nfx/NUkYFhOb/dG0="; sst-main=Sst1|PQHK7-dq45XYrUuGSix2lAlhCRyiW2NWVFuna74UkYXNpjNlyOKDJBCmQr7J0uPV6tgFWaMi6WTaXTfU4fy2q-vDO0hXtckM8V04J1DtQ5fZUJ6Rg7pB10gqDL8NMTYAv7FelKjzdkZ9XYocpzycU2RL8MGj1vWzgD6a6s8-Vl2w7bX4nxLsQWHvFaHkBFaP7FBLIHcy-wQRu7ffgS1R52ivCFL3XUsj8wdEeb0GgUk_Lj2YDHx0KtnODxZCZxmQUzr6cWpIzoMZQ9bHAZ4NpLGy6A-xV1JBbokbWkwVF1Vf9Ns; lc-acbcn=zh_CN; i18n-prefs=CNY; s_cc=true; s_vnum=2090596794466&vn=1; s_sq=[[B]]; s_nr=1658596844424-New; s_dslv=1658596844426; s_ppv=43; session-token=vj9pHfWT4XRBuZ+x6sqqs2aStssRhtp1YERRRqhmmJx+Y5Hcf6McmtFPPiIhkpYt/bMpWj6t1X+gmxqtoeg4Am07DlwqDZT2xw9EYvpDTwJ4EsNqF28EHMa8NMVSiR0vw+dOjJUP3X/AGbf7XGnenYLfleplsD7tFKIaonrCK72tx+BXymPnQx7iazKT0PD6D9DPF+tfo5bPQmL80Cc4kGiEtOxhxIsw; session-id-time=2082729601l; csm-hit=tb:9NCA8D53SB29HAZYH14G+sa-XAZ077H79F9W4YCZ0EJX-1EHD48SB2KK3R9PP4MH1|1658908448718&t:1658908448719&adb:adblk_no'
    }

    resp = requests.get(base_url, headers=headers)
    # print(str(resp))

    content = resp.content.decode('utf-8')
    print(str(content))
    return {}


    select = Selector(text=content)

    # print(str(select))

    productTitle = select.xpath('//*[@id="productTitle"]/text()').extract_first().strip()
    # ags_shipping_import_fee = select.xpath('//*[@id="ags_shipping_import_fee"]/text()').extract_first()
    print("productTitle: " + productTitle)
    # print("ags_shipping_import_fee: "+ags_shipping_import_fee)

    # merchant_id = select.xpath('//*[@id="merchantID"]/text()').extract_first().strip()
    merchant_id = select.css('#merchantID').re_first(
        '<input type="hidden" id="merchantID" name="merchantID" value="(.*?)">')
    # merchant_id = selector.css('.item-0').re_first('<span class="bold">(.*?)</span>')
    print("merchant_id: " + merchant_id)

    # asin_info = {
    #     "productTitle": select.xpath("/text()"),
    #     "ags_shipping_import_fee": select.xpath("/text()")
    # }

    asin_info = {
        "productTitle": select.xpath('//*[@id="productTitle"]/text()').extract_first().strip(),
        # "ags_shipping_import_fee": select.xpath('//*[@id="ags_shipping_import_fee"]/text()').extract_first().strip()
        "ags_shipping_import_fee": ags_shipping_import_fee(asin_no, merchant_id)
    }

    return asin_info


def ags_shipping_import_fee(asin_no: str, merchant_id: str) -> float:
    # base_url: str = 'https://data.amazon.cn/api/marketplaces/AAHKV2X7AFYLW/products/' + asin_no + '?merchantId=' + merchant_id
    base_url: str = 'https://data.amazon.cn/api/marketplaces/AAHKV2X7AFYLW/products/' + asin_no + '?merchantId=A3CQWPW49OI3BQ'
    headers: dict = {
        # "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
        # "Accept": "application/vnd.com.amazon.api+json; type=\"product/v2\"; expand=\"buyingOptions[].importDetails(product.import-details/v1),buyingOptions[].globalStore(product.global-store/v1)\"",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN",
        "DNT":"1",
        "Host":"data.amazon.cn",
        "Origin":"www.amazon.cn",
        "Referer":"www.amazon.cn",
        "X-API-CSRF-TOKEN": "1@g0dZsyVyOIZbol4jMaJqoRZp55qHnLQXHam+x9ynsDQCAAAAAQAAAABi4O9ocmF3AAAAABVX8CwXqz42z+J7i/ABqA==@NLD_5740BY",
        "sec-ch-ua": "\".Not/A)Brand\";v=\"99\", \"Google Chrome\";v=\"103\", \"Chromium\";v=\"103\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "Windows",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",

    "Connection": "keep-alive",
    "Content-Type":"application/json",
    # 'Accept':'*/*',
    # 'Accept-Language': 'zh-CN,zh;q=0.9',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    # 'accept-encoding': 'gzip, deflate, br',
    # # 'accept-language': 'zh-CN,zh;q=0.9',
    # 'cache-control': 'max-age=0',
    # 'downlink': '8',
    # 'ect': '4g',
    # 'rtt': '250',
    # 'Cookie': "session-id=257-3500989-3695223; i18n-prefs=GBP; ubid-acbuk=257-5950834-2508848; x-wl-uid=1bEcLG2b03/1tAwPJNyfuRH+U7J9ZaPYejSBR4HXKuYQPJtLhQbDYyO/GOMypGKXqZrG7qBkS0ng=; session-token=x04EF8doE84tE+6CXYubsjmyob/3M6fdmsQuqzD0jwl/qGdO5aRc2eyhGiwoD0TFzK1rR/yziHsDS4v6cdqT2DySFXFZ9I5OHEtgufqBMEyrA0/Scr87KKA+GWOjfVmKRuPCqOGaixZQ6AIjU3e2iFOdM+3v90NeXFI3cazZcd6x9TYCy9b5u9V8zR7ePbdP; session-id-time=2082758401l; csm-hit=tb:MAA188S1G57TNTH6HQCZ+s-T9EGT4C8FC8J74X5T7CY|1594212767446&t:1594212767446&adb:adblk_no",
    # 'upgrade-insecure-requests': '1',
    # 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
        "User-Agent": UserAgent().random,
        'Cookie': 'session-id=457-4712657-0321906; ubid-acbcn=458-7940515-9599013; x-acbcn="VZU?pPyRMUhNogIOVS?2BX?zroShiKcjr6AXEpGM5If?buubfVZyXWm1dEpdP5CY"; at-main=Atza|IwEBIPqnFPANuSQ54XEOYnk-Wh1HdOKhB0hqCmhiSKdF4v6qLoNrSz1qeaghOPOv-KloCnqFs7gQSe-JDWbrNHM3jX1UeRWzQW_nXFItpl52c1DNrfGCdRJ2iWHk0TQy_L-ENijF7s2vfWuFxtRq38J3a4_QughA92rg3dojEItjQa_nqgqxKTkby7CSZbYs5--PSLHg0GKFUOcmvMQgQluUHgRCzBTUJBR9SxtS2Dc-NHWorA; sess-at-main="vDi1Ir2E4+MsC9cjP92cDaQQ3G9Nfx/NUkYFhOb/dG0="; sst-main=Sst1|PQHK7-dq45XYrUuGSix2lAlhCRyiW2NWVFuna74UkYXNpjNlyOKDJBCmQr7J0uPV6tgFWaMi6WTaXTfU4fy2q-vDO0hXtckM8V04J1DtQ5fZUJ6Rg7pB10gqDL8NMTYAv7FelKjzdkZ9XYocpzycU2RL8MGj1vWzgD6a6s8-Vl2w7bX4nxLsQWHvFaHkBFaP7FBLIHcy-wQRu7ffgS1R52ivCFL3XUsj8wdEeb0GgUk_Lj2YDHx0KtnODxZCZxmQUzr6cWpIzoMZQ9bHAZ4NpLGy6A-xV1JBbokbWkwVF1Vf9Ns; lc-acbcn=zh_CN; i18n-prefs=CNY; s_cc=true; s_vnum=2090596794466&vn=1; s_sq=[[B]]; s_nr=1658596844424-New; s_dslv=1658596844426; s_ppv=43; session-token=rapDMzM4cXIxtLB/v2cGLD8ANRA1sXxDvTV+tGKniGDKsOyvOQSZ7vrPGONw0f62+1Vd+mGwADlv2TlGDAFtotctxE3z9zbpLqMTzCDnatrk78rwXRUdVHFTYlq5mg+pGFdlbRAkvAviw5Kazb1jaxRiJd8fjuIzqgkA/mcsSHF8rwhyZKYht6jnhTlY+6ngdqWe3J7Qabha8WVQBhzm3UK12ycjou3s; session-id-time=2082787201l'

    }

    resp = requests.get(base_url, headers=headers)
    content = resp.content.decode('utf-8')

    print("content:"+content)

    resp_json = json.loads(content)
    ags_shipping_import_fee = resp_json["entity"]['buyingOptions'](0)['importDetails']['entity']['fee']['amount']

    return float(ags_shipping_import_fee)


if __name__ == '__main__':
    product_info: dict = crawl(asin_no='B0050N5DPY')
    print(product_info)


js

In [ ]:
const puppeteer = require('puppeteer')

let main = async () => {
    // 1. 打开浏览器
    const browser = await puppeteer.launch({executablePath:`C:/A-Install/Win_x64_972766_chrome-win/chrome-win/chrome.exe`})
    
    console.log("browser")
    // 2. 新建一个标签页
    const page = await browser.newPage()
    console.log("page")
    await page.setViewport({width:1920,height: 1080})
    // 3. 输入地址敲回车
    await page.goto('https://www.amazon.cn/dp/B007II23XO/ref=z_cn?th=1&psc=1')
    let delivery_span = await page.$eval(`div[id='delivery-block-ags-dcp-container_0']`, e => e.innerText)
    // let delivery_fee = documentHandle.getElementById('delivery-block-ags-dcp-container_0').getElementsByTagName('span')[0].getAttribute('data-csa-c-delivery-price')
    // let add_fee = documentHandle.getElementById('ags_shipping_import_fee').text
    let reg_get_add = /\¥[.\d]*/;
    let add_fee = await page.$eval(`span[id='ags_shipping_import_fee']`, e=> e.outerHTML)
    console.log(`delivery_fee : ${reg_get_add.exec(delivery_span)}, taxation : ${reg_get_add.exec(add_fee)} ; `)
    // 4. 操作：
    //    对加载完毕的网页进行截图
    await page.screenshot({ path: '100016777690.png' })

    // 5. 关闭浏览器
    await browser.close()
}

main()



Submit

In [ ]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


def crawl(asin_no: str) -> dict:
    base_url: str = 'https://www.amazon.cn/dp/' + asin_no + '/ref=z_cn'

    option = Options()
    option.add_experimental_option("detach", True)
    option.add_experimental_option('excludeSwitches', ['enable-automation'])
    option.add_argument('--disable-blink-features=AutomationControlled')

    web = Chrome(options=option)
    web.get(base_url)

    time.sleep(2)

    # 获取商品名
    productTitle: str = web.find_element(By.XPATH, '//*[@id="productTitle"]').text
    time.sleep(2)

    web.find_element(By.XPATH, '//*[@id="native_dropdown_selected_size_name"]').click()
    time.sleep(1)

    web.find_element(By.XPATH, '//*[@id="native_size_name_11"]').click()
    time.sleep(3)

    ags_shipping_import_fee = web.find_element(By.XPATH, '//*[@id="ags_shipping_import_fee"]').text

    asin_info = {
        "productTitle": productTitle,
        "ags_shipping_import_fee": ags_shipping_import_fee
    }

    return asin_info


if __name__ == '__main__':
    product_info: dict = crawl(asin_no='B0050N5DPY')
    print(product_info)
